Imports

In [ ]:
# General
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display

# From a2e2g platform
from a2e2g import a2e2g # module containing wrappers to different components
import a2e2g.modules.market.market as mkrt # market simulator

Settings and directories

In [ ]:
# Data storage location
data_directory = "data"
wind_plant = "staggered_50MW"

# Forecast parameters
utc_minus = 6
tz_current = 'US/Central'
tz_adjust = 18

# Data paths (data not stored in github repository)
wind_data_file = 'a2e2g/'+data_directory+'/measured/wind_obs_20190801-20200727_v2.csv'
hrrr_files = 'a2e2g/'+data_directory+'/forecast/hrrr/*.csv*'

# Configure the market parameters
startTimeSim = "2019-09-28"
stopTimeSim = "2019-09-28"
market_name = "ERCOT"
# all options for ERCOT ["HB_SOUTH","HB_NORTH","HB_WEST", "HB_BUSAVG"]
bus="HB_BUSAVG"

Establish main simulation class that will be used to call modules

In [ ]:
sim = a2e2g.a2e2g(data_directory, wind_plant)

## Day ahead participation

Forecast day ahead atmospheric conditions

In [ ]:
forecast_outputs = sim.forecast(
    utc_minus, tz_current, tz_adjust, wind_data_file, hrrr_files, day=startTimeSim
)

print("\nWind speed:")
display(forecast_outputs[0].head())
print("\nWind direction:")
display(forecast_outputs[1].head())



Estimate day ahead wind plant power production

In [ ]:
floris_outputs = sim.floris_estimation(forecast_outputs, scale_to_MW=True)

display(floris_outputs.head())

Plot day ahead atmospheric and power forecasts

In [ ]:
fig, ax = plt.subplots(3,1,sharex=True)
fig.set_size_inches(10,10)

# Wind speed forecast
ax[0].set_title("Day ahead forecasts")
ax[0].plot(forecast_outputs[0]['Analog Mean'], color='C0')
ax[0].fill_between(forecast_outputs[0].index,
    forecast_outputs[0]['Analog Mean']+forecast_outputs[0]['Analog Stde'],
    forecast_outputs[0]['Analog Mean']-forecast_outputs[0]['Analog Stde'], 
    color='C0', alpha=0.2)
ax[0].set_ylabel('Wind speed [m/s]')

# Wind direction forecast
ax[1].plot(forecast_outputs[1]['Analog Mean'], color='C0')
ax[1].fill_between(forecast_outputs[0].index,
    forecast_outputs[1]['Analog Mean']+forecast_outputs[1]['Analog Stde'],
    forecast_outputs[1]['Analog Mean']-forecast_outputs[1]['Analog Stde'], 
    color='C0', alpha=0.2)
ax[1].set_ylabel('Wind direction [deg]')

# Plot power forecast
ax[2].plot(floris_outputs.Time, (floris_outputs.PWR_MEAN), color='C0')
ax[2].fill_between(floris_outputs.Time,
    (floris_outputs.PWR_MEAN+floris_outputs.PWR_STD),
    (floris_outputs.PWR_MEAN-floris_outputs.PWR_STD),
    color='C0', alpha=0.2)
ax[2].set_ylabel('Plant power [MW]')

Participate in the day ahead market

In [ ]:
# Initialize market object
market = mkrt.Market(startTimeSim, stopTimeSim, market_name, bus, data_directory=data_directory)
dfrt2, dfrt2AS, day_ahead_prices, RTprices = sim.load_price_data(market)

# Determine day ahead market bid to make
day_ahead_bid = sim.day_ahead_bidding(market, dfrt2, dfrt2AS, floris_outputs)

display(day_ahead_bid)

In [ ]:
# use day ahead bid to determine intermediate bid

# if gplk error arises, run cell again!

intermediate_bid = sim.intermediate_bidding(market, day_ahead_bid, day_ahead_prices)
display(intermediate_bid)

## Real-time market

Generate 5-minute ahead forecasts

In [ ]:
# Atmopsheric forecast (simple persistence)
df_st_forecast = sim.short_term_persistence(
    wind_data_file, daterange=(startTimeSim,stopTimeSim)
)

# Power estimate based on atmospheric forecast
short_term_power_estimate = sim.floris_deterministic(df_st_forecast, scale_to_MW=True)

display(df_st_forecast.head())

display(short_term_power_estimate.head())

Add real-time forecast signals to plots

In [ ]:
fig, ax = plt.subplots(3,1,sharex=True)
fig.set_size_inches(10,10)

# Wind speed forecast
ax[0].plot(forecast_outputs[0]['Analog Mean'], color='C0', 
    label='Day ahead')
ax[0].fill_between(forecast_outputs[0].index,
    forecast_outputs[0]['Analog Mean']+forecast_outputs[0]['Analog Stde'],
    forecast_outputs[0]['Analog Mean']-forecast_outputs[0]['Analog Stde'], 
    color='C0', alpha=0.2)
ax[0].plot(df_st_forecast.Time, df_st_forecast.WS, color='red', 
    label='5 mins ahead')
ax[0].set_ylabel('Wind speed [m/s]')
ax[0].legend(loc='upper right')

# Wind direction
ax[1].plot(forecast_outputs[1]['Analog Mean'], color='C0')
ax[1].fill_between(forecast_outputs[0].index,
    forecast_outputs[1]['Analog Mean']+forecast_outputs[1]['Analog Stde'],
    forecast_outputs[1]['Analog Mean']-forecast_outputs[1]['Analog Stde'], 
    color='C0', alpha=0.2)
ax[1].plot(df_st_forecast.Time, df_st_forecast.WD, color='red')
ax[1].set_ylabel('Wind direction [deg]')

# Power
ax[2].plot(floris_outputs.Time, (floris_outputs.PWR_MEAN), color='C0')
ax[2].fill_between(floris_outputs.Time,
    (floris_outputs.PWR_MEAN+floris_outputs.PWR_STD),
    (floris_outputs.PWR_MEAN-floris_outputs.PWR_STD),
    color='C0', alpha=0.2)
ax[2].plot(short_term_power_estimate.Time, (short_term_power_estimate.PWR_MEAN), 
    color='red')
ax[2].set_ylabel('Plant power [MW]')

Participate in real-time market

In [ ]:
RTbid, df_RT_result = sim.real_time_AGC_signal(
    market, intermediate_bid, short_term_power_estimate
)

df_RT_result = market.real_time_market_simulation(df_RT_result, RTbid, RTprices)

display(df_RT_result.head())

## Real-time plant operation

Generate AGC signals

In [ ]:
# System-wide signal
AGC = market.create_system_regulation_signal(create_AGC=True)

# Plant signal
AGC = market.create_wind_plant_regulation_signal(AGC, df_RT_result)

display(AGC.head())

Plot the power signals and AGC signal to follow

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_size_inches(10,4)

# Power
ax.plot(floris_outputs.Time, (floris_outputs.PWR_MEAN), color='C0',
    label='Day ahead')
ax.fill_between(floris_outputs.Time,
    (floris_outputs.PWR_MEAN+floris_outputs.PWR_STD),
    (floris_outputs.PWR_MEAN-floris_outputs.PWR_STD),
    color='C0', alpha=0.2)
ax.plot(short_term_power_estimate.Time, (short_term_power_estimate.PWR_MEAN), 
    color='red', label='5 mins ahead')
ax.set_ylabel('Plant power [MW]')

# Generate time data for plot
agc_times = pd.date_range(
    df_st_forecast.Time.iloc[0]-pd.Timedelta(5, 'm'),
    df_st_forecast.Time.iloc[-1]-pd.Timedelta(4, 's'), 
    freq='4s'
)
AGC['time'] = agc_times
ax.plot(AGC.time, AGC['Basepoint signal'], label='AGC', color="black")
ax.legend()

# Trim down to point of interest
ax.set_xlim([datetime(2019, 9, 28, 5, 45), datetime(2019, 9, 28, 6, 15)])


Simulate control actions

In [ ]:
# Load a test simultion wind case
df_wind = sim.load_test_winds(wind_data_file, daterange=(startTimeSim,stopTimeSim))

# Limit to short period for demonstrative purposes
sim_range = [datetime(2019, 9, 28, 5, 45), datetime(2019, 9, 28, 6, 15)]

# Generate controls, simulate using dynamic FLORIS simulator over a short 
# period with basline power maximizing control ('base') and 
# error-proportional active wind plant power control ('P').
df_sim = sim.simulate_operation(control_cases=['base', 'P'],
    df_wind=df_wind, df_AGC_signal=AGC, closed_loop_simulator='FLORIDyn',
    sim_range=sim_range, dt=1.0)

Plot controller responses

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_size_inches(10,4)

# Power forecast
ax.plot(floris_outputs.Time, (floris_outputs.PWR_MEAN), color='C0',
    label='Day ahead')
ax.fill_between(floris_outputs.Time,
    (floris_outputs.PWR_MEAN+floris_outputs.PWR_STD),
    (floris_outputs.PWR_MEAN-floris_outputs.PWR_STD),
    color='C0', alpha=0.2)
ax.plot(short_term_power_estimate.Time, (short_term_power_estimate.PWR_MEAN), 
    color='red', label='5 mins ahead')
ax.set_ylabel('Plant power [MW]')

# Signal to track
ax.plot(AGC.time, AGC['Basepoint signal'], label='AGC', color="black")

# Plant power output
ax.plot(df_sim.time, df_sim['P_act_base']/1e6, color='blue', label='baseline PMC')
ax.plot(df_sim.time, df_sim['P_act_P']/1e6, color='green', label='A2e2g WPPC')
ax.legend()

# Trim down to point of interest
ax.set_xlim(sim_range)